In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from robustipy.models import OLSRobust
from robustipy.utils import get_colormap_colors
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import seaborn as sns

n_draws = 50

In [11]:
def figure_plotter(results_1,
                   results_2,
                   results_3,
                   results_4,
                   fig_name):

    colors = get_colormap_colors('Spectral_r', 5, brightness_threshold=0.8)
    plt.rcParams['font.family'] = 'Helvetica'
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

    df1a = results_1.summary_df.sort_values('median').reset_index(drop=True)
    df1b = results_2.summary_df.sort_values('median').reset_index(drop=True)
    df2a = results_3.summary_df.sort_values('median').reset_index(drop=True)
    df2b = results_4.summary_df.sort_values('median').reset_index(drop=True)


    ax1.plot(df1a.index, df1a['median'], label='Promotion Condition', color=colors[0])
    ax1.fill_between(df1a.index, df1a['ci_down'], df1a['ci_up'], color=colors[0], alpha=0.15)
    ax1.plot(df1b.index, df1b['median'], label='Prevention Condition', color=colors[3])
    ax1.fill_between(df1b.index, df1b['ci_down'], df1b['ci_up'], color=colors[3], alpha=0.15)
    ax1.set_xlabel('Model Rank', fontsize=16)
    ax1.set_ylabel('Reconstructed Estimate\n(with Moral Impurity)', fontsize=16)
    ax1.set_title('a.', loc='left', fontsize=18, y=1.025, x=-0.075)
    ax1.spines['bottom'].set_position(('outward', 10))
    ax1.spines['left'].set_position(('outward', 10))
    ax1.legend()
    ax1.tick_params(labelsize=12)
    ax1.set_ylim(-0.6, 0.7)
    ax1.set_xlim(0, ax1.get_xlim()[1])
    ax1.grid(True, axis='both', linestyle=(0, (2, 6)),
             linewidth=0.74, alpha=0.3, color='k', zorder=3)

    ax2.plot(df2a.index, df2a['median'], label='Promotion Condition', color=colors[0])
    ax2.fill_between(df2a.index, df2a['ci_down'], df2a['ci_up'], color=colors[0], alpha=0.15)
    ax2.plot(df2b.index, df2b['median'], label='Prevention Condition', color=colors[3])
    ax2.fill_between(df2b.index, df2b['ci_down'], df2b['ci_up'], color=colors[3], alpha=0.15)
    ax2.set_ylabel('Published Estimate\n(with Moral Impurity)', fontsize=16)
    ax2.set_xlabel('Model Rank', fontsize=16)
    ax2.legend()
    ax2.spines['bottom'].set_position(('outward', 10))
    ax2.spines['left'].set_position(('outward', 10))
    ax2.set_title('b.', loc='left', fontsize=18, y=1.025, x=-0.075)
    ax2.tick_params(labelsize=12)
    ax2.set_ylim(-0.6, 0.7)
    ax2.set_xlim(0, ax2.get_xlim()[1])
    ax2.grid(True, axis='both', linestyle=(0, (2, 6)),
             linewidth=0.74, alpha=0.3, color='k', zorder=3)


    legend_elements2 = [Patch(facecolor=colors[0], edgecolor='k',
                              label=r'Promotion (CI)', alpha=0.7),
                        Line2D([0], [0], color=colors[0], lw=1, linestyle='-',
                               label=r'Promotion (Med.)', alpha=0.7),
                        Patch(facecolor=colors[3], edgecolor='k',
                              label=r'Prevention (CI)', alpha=0.7),
                        Line2D([0], [0], color=colors[3], lw=1, linestyle='-',
                               label=r'Prevention (Med.)', alpha=0.7)]
    ax2.legend(handles=legend_elements2, loc='upper right', frameon=True,
               fontsize=10, framealpha=1, facecolor='w', ncols=2,
               edgecolor=(0, 0, 0, 1),
               )

    ax1.get_legend().remove()
    
    plt.tight_layout()
    sns.despine()
    plt.show()
    #plt.savefig("./figures/gino/" + fig_name + '.pdf', bbox_inches='tight')
    #plt.savefig("./figures/gino/" + fig_name + '.png', bbox_inches='tight', dpi=400)
    #plt.savefig("./figures/gino/" + fig_name + '.svg', bbox_inches='tight')

In [12]:
data = pd.read_csv('./data/1_3_prep_mcs_data.csv')

/tmp/ipykernel_19139/728670720.py:1: DtypeWarning: Columns (2098) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./data/1_3_prep_mcs_data.csv')


## No-Controls, participant

In [13]:
y = ["fcmdsa00r",
    "fcmdsb00r",
    "fcmdsc00r",
    "fcmdsd00r",
    "fcmdse00r",
    "fcmdsf00r",
    "fcmdsg00r",
    "fcmdsh00r",
    "fcmdsi00r",
    "fcmdsj00r",
    "fcmdsk00r",
    "fcmdsl00r",
    "fcmdsm00r",
    "fcsati00r",
    "fcgdql00r",
    "fcdowl00r",
    "fcvalu00r",
    "fcgdsf00r",
    "fcscwk00r",
    "fcwylk00r",
    "fcfmly00r",
    "fcfrns00r",
    "fcschl00r",
    "fclife00r",
    ]
x = [
    "tech"
]
c= []

orben_1 = OLSRobust(y=y, x=x, data=data)
orben_1.fit(controls=c, draws=n_draws, composite_sample=100)
orben_1_results = orben_1.get_results()

/home/daniel/code/robustipy/src/robustipy/prototypes.py:128: MissingValueWarning: Missing values found in data. Listwise deletion will be applied.
  warnings.warn(


Calculating Composite Ys


Output()

[OLSRobust] Running with n_cpu=7, draws=50


## Control, Participant

In [14]:
y = ["fcmdsa00r",
    "fcmdsb00r",
    "fcmdsc00r",
    "fcmdsd00r",
    "fcmdse00r",
    "fcmdsf00r",
    "fcmdsg00r",
    "fcmdsh00r",
    "fcmdsi00r",
    "fcmdsj00r",
    "fcmdsk00r",
    "fcmdsl00r",
    "fcmdsm00r",
    "fcsati00r",
    "fcgdql00r",
    "fcdowl00r",
    "fcvalu00r",
    "fcgdsf00r",
    "fcscwk00r",
    "fcwylk00r",
    "fcfmly00r",
    "fcfrns00r",
    "fcschl00r",
    "fclife00r",
    ]
x = [
    "tech",
    "edumot",
    "fd06e00",
    "clpar",
    "fcpaab00",
    "fpwrdscm",
    "fdacaq00",
    "fd05s00",
    "fpclsi00",
    "fpchti00",
    "fdkessl",
    "fdtots00",
    "foede000"
]
c= []

orben_2 = OLSRobust(y=y, x=x, data=data)
orben_2.fit(controls=c, draws=n_draws, composite_sample=100)
orben_2_results = orben_2.get_results()

/home/daniel/code/robustipy/src/robustipy/prototypes.py:128: MissingValueWarning: Missing values found in data. Listwise deletion will be applied.
  warnings.warn(


Calculating Composite Ys


Output()

[OLSRobust] Running with n_cpu=7, draws=50


## No Controls, Primary carers

In [15]:
y = ["fpsdpf00",
    "fpsdro00",
    "fpsdhs00",
    "fpsdsr00",
    "fpsdtt00",
    "fpsdsp00",
    "fpsdor00",
    "fpsdmw00",
    "fpsdhu00",
    "fpsdfs00",
    "fpsdgf00",
    "fpsdfb00",
    "fpsdud00",
    "fpsdlc00",
    "fpsddc00",
    "fpsdnc00",
    "fpsdky00",
    "fpsdoa00",
    "fpsdpb00",
    "fpsdvh00",
    "fpsdst00",
    "fpsdcs00",
    "fpsdgb00",
    "fpsdfe00",
    "fpsdte00"
    ]
x = [
    "tech"
]
c= []

orben_3 = OLSRobust(y=y, x=x, data=data)
orben_3.fit(controls=c, draws=n_draws, composite_sample=100)
orben_3_results = orben_3.get_results()

/home/daniel/code/robustipy/src/robustipy/prototypes.py:128: MissingValueWarning: Missing values found in data. Listwise deletion will be applied.
  warnings.warn(


Calculating Composite Ys


Output()

[OLSRobust] Running with n_cpu=7, draws=50


## Controls, Primary Carers

In [16]:
y = ["fpsdpf00",
    "fpsdro00",
    "fpsdhs00",
    "fpsdsr00",
    "fpsdtt00",
    "fpsdsp00",
    "fpsdor00",
    "fpsdmw00",
    "fpsdhu00",
    "fpsdfs00",
    "fpsdgf00",
    "fpsdfb00",
    "fpsdud00",
    "fpsdlc00",
    "fpsddc00",
    "fpsdnc00",
    "fpsdky00",
    "fpsdoa00",
    "fpsdpb00",
    "fpsdvh00",
    "fpsdst00",
    "fpsdcs00",
    "fpsdgb00",
    "fpsdfe00",
    "fpsdte00"
    ]
x = [
    "tech",
    "edumot",
    "fd06e00",
    "clpar",
    "fcpaab00",
    "fpwrdscm",
    "fdacaq00",
    "fd05s00",
    "fpclsi00",
    "fpchti00",
    "fdkessl",
    "fdtots00",
    "foede000"
]
c= []

orben_4 = OLSRobust(y=y, x=x, data=data)
orben_4.fit(controls=c, draws=n_draws, composite_sample=100)
orben_4_results = orben_4.get_results()

/home/daniel/code/robustipy/src/robustipy/prototypes.py:128: MissingValueWarning: Missing values found in data. Listwise deletion will be applied.
  warnings.warn(


Calculating Composite Ys


Output()

[OLSRobust] Running with n_cpu=7, draws=50


Exception ignored in: <function ResourceTracker.__del__ at 0x7f75abd42200>
Traceback (most recent call last):
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7f8fede3e200>
Traceback (most recent call last):
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/usr/lib64/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x7fbd6e952200>
Traceback (most recent call last):
  File "/usr/lib64/python3.13/multip